In [2]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from tqdm import tqdm, tqdm_notebook
from nltk.stem import LancasterStemmer, PorterStemmer
import preprocess

In [13]:
questions = {
}

questions[1]=preprocess.string_to_analogy("land:excavate::shoal:_,sound salvage dredge survey sidle,dredge")
#questions[2]=preprocess.string_to_analogy("sword:brandish::_:_,")
questions[3]=preprocess.string_to_analogy("volcano:quiescent::talent:_,imperious hyperbolical oblique latent pliant,latent")
questions[4]=preprocess.string_to_analogy("style:flamboyant::behavior:_,brazen lofty volatile insolent sassy,brazen")
questions[5]=preprocess.string_to_analogy("direct:confront::oblique:_,unsettle incite sidle stymie flourish,sidle")
questions[6]=preprocess.string_to_analogy("beforehand:trepidation::afterwards:_,bravado decadence hyperbole foolhardy rue,rue")
questions[7]=preprocess.string_to_analogy("person:odious::action:_,unsettling imperious heinous lofty haughty,heinous")
questions[8]=preprocess.string_to_analogy("naughtiness:permit::misbehavior:_,dredge confront pique countenance stymie,countenance")
questions[9]=preprocess.string_to_analogy("speech:hyperbole::behavior:_,trepidation quiescence bravado pique parsimony,bravado")
questions[10]=preprocess.string_to_analogy("emotional:resistance::physical:_,cadaver survey conflagration decadence friction,friction")
questions[11]=preprocess.string_to_analogy("interest:rouse::curiosity:_,assuage nettle dredge rue pique,pique")
questions[12]=preprocess.string_to_analogy("mien:haughty::countenance:_, flamboyant imperious befuddled canny brazen,imperious")
questions[13]=preprocess.string_to_analogy("threateningly:brandish::flamboyantly:_,confront flaunt fan incite garner,flaunt")
#questions[14]=preprocess.string_to_analogy
questions[15]=preprocess.string_to_analogy("latent:possibility::hidden:_,cache ire hovel visage urchin,cache")
#questions[16]=preprocess.string_to_analogy
questions[17]=preprocess.string_to_analogy("travel:blocked::attempt:_,rued incited salvage stymied unsettled,stymied")
questions[18]=preprocess.string_to_analogy("salesman:canny::businesswoman:_,entrepreneurial prolific shrewd haughty frugal,shrewd")
questions[19]=preprocess.string_to_analogy("brave:reckless::frugal:_,prolific audacious foolhardy poor parsimonious,parsimonious")
questions[20]=preprocess.string_to_analogy("personality:pliant::behavior:_,lofty insolent cooperative volatile popular,cooperative")



['land', 'excavate', 'shoal', ['sound', 'salvage', 'dredge', 'survey', 'sidle'], 'dredge']
['volcano', 'quiescent', 'talent', ['imperious', 'hyperbolical', 'oblique', 'latent', 'pliant'], 'latent']
['style', 'flamboyant', 'behavior', ['brazen', 'lofty', 'volatile', 'insolent', 'sassy'], 'brazen']
['direct', 'confront', 'oblique', ['unsettle', 'incite', 'sidle', 'stymie', 'flourish'], 'sidle']
['beforehand', 'trepidation', 'afterwards', ['bravado', 'decadence', 'hyperbole', 'foolhardy', 'rue'], 'rue']
['person', 'odious', 'action', ['unsettling', 'imperious', 'heinous', 'lofty', 'haughty'], 'heinous']
['naughtiness', 'permit', 'misbehavior', ['dredge', 'confront', 'pique', 'countenance', 'stymie'], 'countenance']
['speech', 'hyperbole', 'behavior', ['trepidation', 'quiescence', 'bravado', 'pique', 'parsimony'], 'bravado']
['emotional', 'resistance', 'physical', ['cadaver', 'survey', 'conflagration', 'decadence', 'friction'], 'friction']
['interest', 'rouse', 'curiosity', ['assuage', 'ne

In [3]:
from gensim.parsing.porter import PorterStemmer
import binascii

file = open('./GoogleNews-vectors-negative300.bin','rb')
binascii.b2a_base64(file.read())
#p = PorterStemmer()
#p.stem_documents(file)


MemoryError: 

In [14]:
file = './GoogleNews-vectors-negative300.bin'

#print(f"Training from {file}")
print("Training model...")
model = KeyedVectors.load_word2vec_format(file, binary=True)
model.init_sims(replace=True)
print("Model trained")

Training model...
Model trained


In [15]:
model.vocab

{'</s>': <gensim.models.keyedvectors.Vocab at 0x2604f454f28>,
 'in': <gensim.models.keyedvectors.Vocab at 0x2604f454fd0>,
 'for': <gensim.models.keyedvectors.Vocab at 0x2604f454f98>,
 'that': <gensim.models.keyedvectors.Vocab at 0x2604f454eb8>,
 'is': <gensim.models.keyedvectors.Vocab at 0x2604f454e80>,
 'on': <gensim.models.keyedvectors.Vocab at 0x2604f4533c8>,
 '##': <gensim.models.keyedvectors.Vocab at 0x260361456d8>,
 'The': <gensim.models.keyedvectors.Vocab at 0x2604f45a860>,
 'with': <gensim.models.keyedvectors.Vocab at 0x2604f45a8d0>,
 'said': <gensim.models.keyedvectors.Vocab at 0x2604f45a0b8>,
 'was': <gensim.models.keyedvectors.Vocab at 0x2604f45a4a8>,
 'the': <gensim.models.keyedvectors.Vocab at 0x2604f45a828>,
 'at': <gensim.models.keyedvectors.Vocab at 0x2604f45a668>,
 'not': <gensim.models.keyedvectors.Vocab at 0x2604f45a160>,
 'as': <gensim.models.keyedvectors.Vocab at 0x2604f45a2e8>,
 'it': <gensim.models.keyedvectors.Vocab at 0x2604f45ab00>,
 'be': <gensim.models.keyed

In [4]:
import sys
import numpy as np 

def test(classifiers=["simple"]):
    results = {}
    for classifier in tqdm(classifiers):
        results[classifier] = np.empty((1,0), dtype=int)
        for question_number in tqdm(questions):
            data = questions[question_number] 
            first, second, third, answers, correct = data
            if solve(first, second, third, answers, classifier=classifier) == correct:
                results[classifier] = np.append(results[classifier], [1])
            else:
                results[classifier] = np.append(results[classifier], [0])
    return results

def solve(first, second, third, answers, classifier="simple", verbose=False):
    for answer in answers:
        if answer not in model.vocab:
            print("%s is not in vocab!", answer)
            answers.remove(answer)
            
    if classifier == "simple":
        prediction = model.most_similar_cosmul(positive=[second, third], negative=[first], topn=1)[0][0]
        prediction = model.most_similar_to_given(prediction, answers)
    elif classifier == "minus_third":
        prediction = model.most_similar_cosmul(positive=[second, first], negative=[third], topn=1)[0][0]
        prediction = model.most_similar_to_given(prediction, answers)
    elif classifier == "deep":
        top_predictions = list(zip(*model.most_similar_cosmul(positive=["king", "woman"], negative=["man"], topn=5)))[0]
        best = -1
        for pred in top_predictions:
            for answer in answers:
                similarity = model.similarity(answer,pred)
                if similarity > best:
                    best = similarity
                    prediction = answer
    elif classifier == "deep_uniq":
        top_predictions = list(zip(*model.most_similar_cosmul(positive=["king", "woman"], negative=["man"], topn=5)))[0]
        best = [] 
        for pred in top_predictions:
            best.append(model.most_similar_to_given(pred,answers))
        prediction = max(best, key=best.count)
    elif classifier == "magnitude":
        best = sys.maxsize
        prediction = ""
        
        word_sum = model[second] + model[third] - model[first]
        for answer in answers:
            answer_sum = word_sum - model[answer]
            magnitude = answer_sum.dot(answer_sum)
            if magnitude < best:
                best = magnitude
                prediction = answer
    return prediction 
            
        

In [5]:
results = test(["deep","deep_uniq","simple","magnitude"])

  6%|███████████▊                                                                                                                                                                                            | 1/17 [00:01<00:18,  1.16s/it]

%s is not in vocab! hyperbolical



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:03<00:00, 14.59s/it]


In [6]:
for agent in results:
    print(str(results[agent]) + " " + agent + " " + str(np.sum(results[agent])))
print(results)

[0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0] deep 4
[1 0 0 1 0 0 1 0 0 0 1 1 0 0 0 1 0] deep_uniq 6
[1 0 0 0 0 1 0 1 1 0 0 1 0 1 1 1 0] simple 8
[1 1 0 1 0 1 0 0 1 0 1 1 1 1 1 1 0] magnitude 11
{'deep': array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0]), 'deep_uniq': array([1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0]), 'simple': array([1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0]), 'magnitude': array([1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0])}


In [7]:
results["simple"] + results["magnitude"]

array([2, 1, 0, 1, 0, 2, 0, 1, 2, 0, 1, 2, 1, 2, 2, 2, 0])

In [8]:
e = np.empty((1,0), dtype=int)
e = np.append(e, [0])
e = np.append(e, [0])
e

array([0, 0])

In [ ]:
from sklearn.neighbors import KDTree
class ANNSearch:
    word2idx = {}
    idx2word = {}
    data = []

    def __init__(self, model):
        for counter, key in enumerate(model.vocab.keys()):
            self.data.append(model[key])
            self.word2idx[key] = counter
            self.idx2word[counter] = key

        # leaf_size is a hyperparameter
        self.data = np.array(self.data)
        self.tree = KDTree(self.data, leaf_size=100)
        
    def search_by_vector(self, v, k=10):
        dists, inds = self.tree.query([v], k)
        return zip(dists[0], [self.idx2word[idx] for idx in inds[0]])

    def search(self, query, k=10):
        vector = self.data[self.word2idx[query]]
        return self.search_by_vector(vector, k)